1. Повнозв'язані нейронні мережі  
Вирішіть завдання класифікації даних, з якими ви працювали в лабораторній № 1 за допомогою повнозв’язаної нейромережі прямого поширення (fully connected feed-forward network). Результати порівняйте з одержаними раніше.  

In [23]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, accuracy_score

In [24]:
#зчитуємо дані та нормалізуємо їх
file_path = '../japanese+credit+screening/crx.data'
data = pd.read_csv(file_path)
print(f"Columns names:\n\t{data.columns.tolist()}")
print(f"{data.shape[0]} rows\n{data.shape[1]} columns")

label_encoder = LabelEncoder()
for column in data.columns:
    if data[column].dtype == 'object':
        data[column] = label_encoder.fit_transform(data[column].astype(str))

scaler = StandardScaler()
numerical_columns = ['A2','A7','A15']
data[numerical_columns] = scaler.fit_transform(data[numerical_columns])
print(data.head())

Columns names:
	['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'target']
690 rows
16 columns
   A1        A2     A3  A4  A5  A6        A7    A8  A9  A10  A11  A12  A13  \
0   2  0.056919  0.000   2   1  13  0.773655  1.25   1    1    1    0    0   
1   1  1.846363  4.460   2   1  11 -0.769183  3.04   1    1    6    0    0   
2   1 -0.640132  0.500   2   1  11 -0.769183  1.50   1    0    0    0    0   
3   2 -0.265597  1.540   2   1  13  0.773655  3.75   1    1    5    1    0   
4   2 -1.118704  5.625   2   1  13  0.773655  1.71   1    0    0    0    2   

   A14       A15  target  
0   68 -0.195413       0  
1   11 -0.087852       0  
2   96 -0.037144       0  
3   31 -0.194837       0  
4   37 -0.195413       0  


In [25]:
#розділяємо дані на тренувальні та тестові, описуємо нейрону мережу
target_col = "target"
X = data.drop(columns=[target_col])
y = data[target_col]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3)

model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'), 
    # relu активаційна функція яка допомагає моделі вчитися нелінійним залежностям
    Dropout(0.3),
    # відключенння нейронів для уникнення перенавчання
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  
    # activation='sigmoid': Активаційна функція сигмоїда перетворює вихідне значення у діапазон від 0 до 1
])

/home/user/Documents/data_a/3/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [26]:
#компілюємо модель
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Adam налаштовує швидкість навчання для кожного параметра, що робить його ефективним у широкому діапазоні задач.
# learning_rate=0.001 наскільки сильно модель оновлює ваги
# loss визначає функцію втрат, яка використовується для оцінки того, наскільки добре модель виконує свої передбачення.
# binary_crossentropy є типовою функцією втрат для задач бінарної класифікації

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32, verbose=0)

y_pred = (model.predict(X_test) > 0.6).astype(int)
#якщо > 0.5 то так, якщо меньше то ні
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)


print(accuracy)
print(classification_rep)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
0.7341040462427746
              precision    recall  f1-score   support

           0       0.71      0.76      0.73        83
           1       0.76      0.71      0.74        90

    accuracy                           0.73       173
   macro avg       0.73      0.74      0.73       173
weighted avg       0.74      0.73      0.73       173



Минулі результати:  
  
           0       0.82      0.80      0.81        70  
           1       0.80      0.82      0.81        68  

2. Згорткові нейронні мережі  
Вирішіть завдання класифікації зображень за допомогою згорткової (convolutional) нейромережі двома способами  
а) навчить мережу з нуля (from scratch)  

Для того щоб виконати це завдання було викоритано додатковий код який роділив зображення на тренувальну та тестову частини:
```
import os
import shutil
from sklearn.model_selection import train_test_split

original_dir = "cats_vs_dogs_dataset"
base_dir = "cats_vs_dogs"

os.makedirs(os.path.join(base_dir, "train/cat"), exist_ok=True)
os.makedirs(os.path.join(base_dir, "train/dog"), exist_ok=True)
os.makedirs(os.path.join(base_dir, "test/cat"), exist_ok=True)
os.makedirs(os.path.join(base_dir, "test/dog"), exist_ok=True)

def split_data(source, train_dest, test_dest, test_size=0.2):
    filenames = os.listdir(source)
    train_filenames, test_filenames = train_test_split(filenames, test_size=test_size, random_state=3)
    for filename in train_filenames:
        shutil.copy(os.path.join(source, filename), train_dest)
    for filename in test_filenames:
        shutil.copy(os.path.join(source, filename), test_dest)

split_data(os.path.join(original_dir, "cat"), os.path.join(base_dir, "train/cat"), os.path.join(base_dir, "test/cat"))
split_data(os.path.join(original_dir, "dog"), os.path.join(base_dir, "train/dog"), os.path.join(base_dir, "test/dog"))
```


In [27]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [28]:
# Підготовка даних
train_dir = "cats_vs_dogs/train"
test_dir = "cats_vs_dogs/test"

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(150, 150), batch_size=32, class_mode='binary', subset='training')
val_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(150, 150), batch_size=32, class_mode='binary', subset='validation')

Found 13834 images belonging to 2 classes.
Found 3458 images belonging to 2 classes.


In [29]:
# Модель CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    # 32 фільтри розміром 3 на 3
    MaxPooling2D(2, 2),
    # блок 2 x 2 замінюються на один (максимальне)
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    # Перетворює багатовимірний тензор (вихід попередніх згорткових шарів) у плоский вектор
    Dense(512, activation='relu'),  
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

/home/user/Documents/data_a/3/.venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [30]:
# Компіляція та тренування
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator, validation_data=val_generator, epochs=7)

# Оцінка на тестових даних
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir, target_size=(150, 150), batch_size=32, class_mode='binary')

test_loss, test_acc = model.evaluate(test_generator)
print(f"Точність на тестових даних: {test_acc:.2f}")

/home/user/Documents/data_a/3/.venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/7
278/433 ━━━━━━━━━━━━━━━━━━━━ 2:27 950ms/step - accuracy: 0.5254 - loss: 0.7657

KeyboardInterrupt: 

б) застосуйте перенесення навчання (transfer learning from pre-trained weights)  

In [32]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [33]:
# Підготовка даних
train_dir = "cats_vs_dogs/train"
test_dir = "cats_vs_dogs/test"

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(150, 150), batch_size=32, class_mode='binary', subset='training')
val_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(150, 150), batch_size=32, class_mode='binary', subset='validation')

Found 13834 images belonging to 2 classes.
Found 3458 images belonging to 2 classes.


In [34]:
# Завантаження попередньо натренованої моделі VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
# Заморожування базової моделі
base_model.trainable = False

In [35]:
# Додавання власних шарів
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

# Повна модель
model = Model(inputs=base_model.input, outputs=predictions)

# Компіляція моделі
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [36]:
# Повна модель
model = Model(inputs=base_model.input, outputs=predictions)

# Компіляція моделі
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Початкове навчання верхніх шарів
history = model.fit(train_generator, validation_data=val_generator, epochs=7)

# Розморожування базової моделі (тонке налаштування)
base_model.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Тонке налаштування
history_fine_tune = model.fit(train_generator, validation_data=val_generator, epochs=5)

Epoch 1/7
  6/433 ━━━━━━━━━━━━━━━━━━━━ 21:17 3s/step - accuracy: 0.6135 - loss: 0.6914

KeyboardInterrupt: 

In [ ]:
# Оцінка на тестових даних
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir, target_size=(150, 150), batch_size=32, class_mode='binary')

test_loss, test_acc = model.evaluate(test_generator)
print(f"Точність на тестових даних (Transfer Learning): {test_acc:.2f}")

3. Рекурентні нейронні мережі
Вирішіть задачу класифікації текстів (з якими ви працювали в лабораторній № 2) за допомогою рекурентної нейромережі двома способами:
а) навчить мережу і embedding шар з нуля (from scratch)
б) використовуючи pretrained word embeddings

In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout, LSTM
from sklearn.model_selection import train_test_split

2024-12-21 09:52:54.410850: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-21 09:52:54.954518: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-21 09:52:55.481340: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734767576.006692   11164 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734767576.158328   11164 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-21 09:52:57.343271: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [3]:
# отримуємо дані та нормалізуємо їх
df = pd.read_csv('FakeNewsNet.csv')
texts = df['title'].astype(str).values
labels = df['real'].values

tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
max_len = 50
X = pad_sequences(sequences, maxlen=max_len)
y = np.array(labels)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=3)


In [4]:
# Модель з Embedding з нуля
model_scratch = Sequential()
model_scratch.add(Embedding(input_dim=10000, output_dim=100, input_length=max_len))
model_scratch.add(LSTM(64, return_sequences=False))
model_scratch.add(Dropout(0.2))
model_scratch.add(Dense(1, activation='sigmoid'))
model_scratch.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_scratch.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=64)
scratch_score = model_scratch.evaluate(X_val, y_val, verbose=0)

/home/user/Documents/data_a/3/.venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2024-12-21 09:53:07.987472: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/5
290/290 ━━━━━━━━━━━━━━━━━━━━ 23s 64ms/step - accuracy: 0.7671 - loss: 0.5277 - val_accuracy: 0.8343 - val_loss: 0.3825
Epoch 2/5
290/290 ━━━━━━━━━━━━━━━━━━━━ 18s 62ms/step - accuracy: 0.8785 - loss: 0.2962 - val_accuracy: 0.8386 - val_loss: 0.3881
Epoch 3/5
290/290 ━━━━━━━━━━━━━━━━━━━━ 18s 64ms/step - accuracy: 0.9133 - loss: 0.2206 - val_accuracy: 0.8276 - val_loss: 0.4065
Epoch 4/5
290/290 ━━━━━━━━━━━━━━━━━━━━ 18s 63ms/step - accuracy: 0.9359 - loss: 0.1674 - val_accuracy: 0.8269 - val_loss: 0.4786
Epoch 5/5
290/290 ━━━━━━━━━━━━━━━━━━━━ 18s 63ms/step - accuracy: 0.9528 - loss: 0.1259 - val_accuracy: 0.8209 - val_loss: 0.5475


In [5]:
# Модель з використанням попередньо навчених ембедингів (наприклад, GloVe)
embeddings_index = {}
with open('glove.6B/glove.6B.200d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

embedding_dim = 200
word_index = tokenizer.word_index
embedding_matrix = np.zeros((10000, embedding_dim))
for word, i in word_index.items():
    if i < 10000:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

model_pretrained = Sequential()
embedding_layer = Embedding(input_dim=10000, output_dim=embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False)
model_pretrained.add(embedding_layer)
model_pretrained.add(GRU(64, return_sequences=False))
model_pretrained.add(Dropout(0.2))
model_pretrained.add(Dense(1, activation='sigmoid'))
model_pretrained.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_pretrained.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=64)
pretrained_score = model_pretrained.evaluate(X_val, y_val, verbose=0)

Epoch 1/5
290/290 ━━━━━━━━━━━━━━━━━━━━ 20s 58ms/step - accuracy: 0.7639 - loss: 0.5164 - val_accuracy: 0.8144 - val_loss: 0.4279
Epoch 2/5
290/290 ━━━━━━━━━━━━━━━━━━━━ 16s 57ms/step - accuracy: 0.8252 - loss: 0.4027 - val_accuracy: 0.8259 - val_loss: 0.4082
Epoch 3/5
290/290 ━━━━━━━━━━━━━━━━━━━━ 16s 56ms/step - accuracy: 0.8357 - loss: 0.3785 - val_accuracy: 0.8267 - val_loss: 0.3996
Epoch 4/5
290/290 ━━━━━━━━━━━━━━━━━━━━ 17s 57ms/step - accuracy: 0.8483 - loss: 0.3561 - val_accuracy: 0.8319 - val_loss: 0.3898
Epoch 5/5
290/290 ━━━━━━━━━━━━━━━━━━━━ 17s 58ms/step - accuracy: 0.8548 - loss: 0.3359 - val_accuracy: 0.8388 - val_loss: 0.3885


In [6]:


print("From scratch Embedding Accuracy:", scratch_score[1])
print("Pretrained Embedding Accuracy:", pretrained_score[1])

From scratch Embedding Accuracy: 0.8209051489830017
Pretrained Embedding Accuracy: 0.8387930989265442
